In [182]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model="llama3:8b"):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": 0.2,
        "max_new_tokens": 50 # only when stream = False work
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"
    payload_json = json.dumps(payload)

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)

# Example usage
# answer_this_prompt("What is the Big Bang theory?", stream=True)
# print("------------------------")

In [135]:
from pgmpy.models import BayesianNetwork
from pgmpy.utils import get_example_model

asia = get_example_model('asia')
# print(asia.nodes())
print(asia.edges())
# print(asia.get_cpds())

def query_bn(bn, query_variables, evidence):
    """
    Perform exact inference on a Bayesian Network (BN) given the query variables and evidence.
    
    Args:
    - bn: A Bayesian Network object (pgmpy.models.BayesianNetwork).
    - query_variables: A list of strings specifying the query variables.
    - evidence: A dictionary where keys are strings of the evidence variables and values are the observed states.
    
    Returns:
    - A list of tuples where each tuple contains the state and probability of the query variable.
    """
    # Perform exact inference using Variable Elimination
    from pgmpy.inference import VariableElimination
    inference = VariableElimination(bn)
    result = inference.query(variables=query_variables, evidence=evidence)
    return result

# Example usage
query_variables = ['xray']
evidence = {'bronc': 'yes', 'either': 'yes'}
result = query_bn(asia, query_variables, evidence)
string = str(result)
print(string)


[('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')]
+-----------+-------------+
| xray      |   phi(xray) |
+===========+=============+
| xray(yes) |      0.9800 |
+-----------+-------------+
| xray(no)  |      0.0200 |
+-----------+-------------+


In [43]:
print("\nSimplified CPDs in the Asia model:")
cpd_map = {}
for cpd in asia.get_cpds():
    # print(f"Variable: {cpd.variable}")
    # print(f"Values: {cpd.values.tolist()}")
    cpd_map[cpd.variable] = cpd.values.tolist()
    # print()

# print(cpd_map)


Simplified CPDs in the Asia model:
{'asia': [0.01, 0.99], 'bronc': [[0.6, 0.3], [0.4, 0.7]], 'dysp': [[[0.9, 0.8], [0.7, 0.1]], [[0.1, 0.2], [0.3, 0.9]]], 'either': [[[1.0, 1.0], [1.0, 0.0]], [[0.0, 0.0], [0.0, 1.0]]], 'lung': [[0.1, 0.01], [0.9, 0.99]], 'smoke': [0.5, 0.5], 'tub': [[0.05, 0.01], [0.95, 0.99]], 'xray': [[0.98, 0.05], [0.02, 0.95]]}


In [63]:
PROMPT = """I would like to answer the information to make an inference on a Bayesian network. It is necessary to detect the variables of interest and the evidence presented in a question. The evidence is binary. Only include evidence which is present on the question. The variables of interest should not be in the evidence. Ensure that the evidence and the variables of interest are present in the list of variables. Use the following format: <BEGIN> Question: "Question here" Variables: "List of variables here" Variable of Interest: "List with the interest variables here" Evidences: "Complete JSON with evidences here" <END>|"""
QUESTION = "What is the probability of a person having bronchits given they smoke?"
for i in range(1):
  answer_this_prompt('This is bayesian network:' + str(asia.edges()) + ' ' + PROMPT + ' Question: ' + QUESTION)
  print('---------------------------------------------')

To answer this question, we need to identify the variables of interest and the evidence presented in the question.

Variables of Interest:
The variable of interest is "bronc", which represents Bronchitis.

Evidence:
The evidence presented in the question is "smoke" = 1 (True). This means that the person smokes.

Here's the format you requested:

<BEGIN>
Question: What is the probability of a person having bronchits given they smoke?
Variables: ['asia', 'tub', 'either', 'smoke', 'lung', 'bronc', 'dysp', 'xray']
Variable of Interest: ['bronc']
Evidences: {"smoke": 1}
<END>

---------------------------------------------


## Generate Questions

In [157]:
# PROMPT = """I would like to answer the information to make an inference on a Bayesian network. It is necessary to detect the variables of interest and the evidence presented in a question. The evidence is binary. Only include evidence which is present on the question. The variables of interest should not be in the evidence. Ensure that the evidence and the variables of interest are present in the list of variables. Use the following format: <BEGIN> Question: "Question here" Variables: "List of variables here" Variable of Interest: "List with the interest variables here" Evidences: "Complete JSON with evidences here" <END>|"""
question_list = []
QUESTION = "Give me a short human-like question to ask about this Bayesian network. Just question, no context. For example: What is the probability of a person having bronchitis given they smoke?"
for i in range(10):
  print(i + 1, end=' ')
  question = answer_this_prompt('This is bayesian network:' + str(asia.edges()) + '. Question: ' + QUESTION)
  question_list.append(question)
  # print(ques)
  # print('---------------------------------------------')

print()
print(question_list)

1 2 3 4 5 6 7 8 9 10 
['What is the likelihood that someone has an X-ray if they have symptoms of dyspnea?', "What's the likelihood that someone has pneumonia if they had an x-ray and didn't have dyspnea?", "What's the likelihood that someone has pneumonia if their chest X-ray shows signs of disease?", 'What is the likelihood that a person with abnormal lung function also has dyspnea?', "What's the likelihood that someone has tuberculosis if we know Asia didn't have lung cancer?", "What's the likelihood that someone has dyspnea if their X-ray shows no signs of lung disease or infection?", 'What is the likelihood that someone has tuberculosis if they have an X-ray result indicating either lung or bronchial disease?', "What is the likelihood that a person has lung cancer if an X-ray shows signs of disease and they don't have tuberculosis?", 'What is the probability that a person has X-ray evidence of disease if they also have difficulty breathing?', 'What is the likelihood that someone h

In [166]:
def parse_input(input_string):
    result = {"variables_of_interest": [], "evidence": {}}

    try:
        parts = input_string.split(';')
        for part in parts:
            key, value = part.split(':', 1)  
            key = key.strip()
            value = value.strip()
            
            if key == "variables_of_interest":
                result["variables_of_interest"] = [var.strip().strip("'") for var in value.strip('[]').split(',')]
            
            elif key == "evidence":
                value = value.strip('{}')
                items = value.split(',')
                for item in items:
                    var, val = item.split(':')
                    result["evidence"][var.strip().strip("'")] = val.strip().strip("'")

    except ValueError as e:
        return f"Error parsing input: {e}"
    
    return result

In [167]:
from collections import Counter

def check_consistency(results):
    """
    Check the consistency of results by comparing each result to the majority result.
    
    Args:
        results (list): A list of parsed results (dictionaries).
    
    Returns:
        float: The percentage of consistent results.
        int: The number of inconsistent results.
        dict: The majority result.
    """
    if not results:
        print("No results to check.")
        return 0.0, 0, None

    # Find the majority result using Counter
    result_counts = Counter(map(str, results))  # Convert dictionaries to strings for counting
    majority_result_str, _ = result_counts.most_common(1)[0]
    majority_result = eval(majority_result_str)  # Convert the string back to a dictionary

    # Count inconsistencies
    inconsistent_count = sum(1 for result in results if result != majority_result)
    total_results = len(results)
    consistent_count = total_results - inconsistent_count
    consistency_percentage = (consistent_count / total_results) * 100

    return consistency_percentage, inconsistent_count, majority_result

In [172]:
PROMPT = r"I would like to answer the information to make an inference on a Bayesian Network. It is necessary to detect the variables of interest and the evidence presented in a question. The evidence is binary (yes, no). Only include evidence which is present on the question. The variables of interest should not be in the evidence. No explanation, no prediction of probability, only give short answer in this format, with a semicolon seperate them like this: variables_of_interest: []; evidence: {}"

consistency_list = []

for i, question in enumerate(question_list):
      print('Question', i + 1, ':', question)
      rs = []
      for i in range(10):
            print(i+1, end=' ')
            response = answer_this_prompt('This is bayesian network:' + str(asia.edges()) + ' ' + PROMPT + ' Question: ' + question)

            result = parse_input(response)
            # print('llm result\n', result, end='\n\n')

            # string_result = str(query_bn(asia, result["variables_of_interest"], result["evidence"]))
            # # print('bn result\n', string_result, end='\n\n')

            # answer = answer_this_prompt('Based on this result: ' + string_result + '. Give a short answer this question: ' + QUESTION)
            # # print('\nllm answer based on bn result:\n', answer)
            # print(result)
            rs.append(result)
            # print(answer)
      consistency_percentage, inconsistent_count, majority_result = check_consistency(rs)
      consistency_list.append(consistency_percentage)
      print(f"\nConsistency: {consistency_percentage:.2f}%")
      print(f"Inconsistent results: {inconsistent_count}")
      print(f"Majority result: {majority_result}")
      print('---------------------------------------------')
      print()

print(f'Average consistency: {sum(consistency_list) / len(consistency_list)}%')

Question 1 : What is the likelihood that someone has an X-ray if they have symptoms of dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 70.00%
Inconsistent results: 3
Majority result: {'variables_of_interest': ['xray'], 'evidence': {'dysp': 'yes'}}
---------------------------------------------

Question 2 : What's the likelihood that someone has pneumonia if they had an x-ray and didn't have dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 70.00%
Inconsistent results: 3
Majority result: {'variables_of_interest': ['pneumonia'], 'evidence': {'xray': 'yes', 'dysp': 'no'}}
---------------------------------------------

Question 3 : What's the likelihood that someone has pneumonia if their chest X-ray shows signs of disease?
1 2 3 4 5 6 7 8 9 10 
Consistency: 80.00%
Inconsistent results: 2
Majority result: {'variables_of_interest': ['pneumonia'], 'evidence': {'xray': 'yes'}}
---------------------------------------------

Question 4 : What is the likelihood that a person with abnormal lung function al

### More aware of BN rule of Common Cause, Common Effect, and Chain Rule

In [173]:
PROMPT = r"I would like to answer the information to make an inference on a Bayesian Network. It is necessary to detect the variables of interest and the evidence presented in a question. The evidence is binary (yes, no). Only include evidence which is present on the question. The variables of interest should not be in the evidence. Be aware of Common Cause, Common Effect, and Chain Rule in Bayesian Network. No explanation, no prediction of probability, only give short answer in this format, with a semicolon seperate them like this: variables_of_interest: []; evidence: {}"

consistency_list = []

for i, question in enumerate(question_list):
      print('Question', i + 1, ':', question)
      rs = []
      for i in range(10):
            print(i+1, end=' ')
            response = answer_this_prompt('This is bayesian network:' + str(asia.edges()) + ' ' + PROMPT + ' Question: ' + question)

            result = parse_input(response)
            # print('llm result\n', result, end='\n\n')

            # string_result = str(query_bn(asia, result["variables_of_interest"], result["evidence"]))
            # # print('bn result\n', string_result, end='\n\n')

            # answer = answer_this_prompt('Based on this result: ' + string_result + '. Give a short answer this question: ' + QUESTION)
            # # print('\nllm answer based on bn result:\n', answer)
            # print(result)
            rs.append(result)
            # print(answer)
      consistency_percentage, inconsistent_count, majority_result = check_consistency(rs)
      consistency_list.append(consistency_percentage)
      print(f"\nConsistency: {consistency_percentage:.2f}%")
      print(f"Inconsistent results: {inconsistent_count}")
      print(f"Majority result: {majority_result}")
      print('---------------------------------------------')
      print()

print(f'Average consistency: {sum(consistency_list) / len(consistency_list)}%')

Question 1 : What is the likelihood that someone has an X-ray if they have symptoms of dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 100.00%
Inconsistent results: 0
Majority result: {'variables_of_interest': ['xray'], 'evidence': {'dysp': 'yes'}}
---------------------------------------------

Question 2 : What's the likelihood that someone has pneumonia if they had an x-ray and didn't have dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 40.00%
Inconsistent results: 6
Majority result: {'variables_of_interest': ['pneumonia'], 'evidence': {'xray': 'yes', 'dysp': 'no'}}
---------------------------------------------

Question 3 : What's the likelihood that someone has pneumonia if their chest X-ray shows signs of disease?
1 2 3 4 5 6 7 8 9 10 
Consistency: 50.00%
Inconsistent results: 5
Majority result: {'variables_of_interest': ['pneumonia'], 'evidence': {'xray': 'yes'}}
---------------------------------------------

Question 4 : What is the likelihood that a person with abnormal lung function a

In [184]:
QUESTION = "What is the probability of a person having bronchitis given they smoke?"
response = answer_this_prompt('This is bayesian network:' + str(asia.edges()) + ' ' + PROMPT + ' Question: ' + QUESTION, model="phi4:latest", stream=True)

variables_of_interest: ['bronc'];  
evidence: {'smoke': 'yes'}

#### phi4:latest

In [185]:
PROMPT = r"I would like to answer the information to make an inference on a Bayesian Network. It is necessary to detect the variables of interest and the evidence presented in a question. The evidence is binary (yes, no). Only include evidence which is present on the question. The variables of interest should not be in the evidence. Be aware of Common Cause, Common Effect, and Chain Rule in Bayesian Network. No explanation, no prediction of probability, only give short answer in this format, with a semicolon seperate them like this: variables_of_interest: []; evidence: {}"

consistency_list = []

for i, question in enumerate(question_list):
      print('Question', i + 1, ':', question)
      rs = []
      for i in range(10):
            print(i+1, end=' ')
            response = answer_this_prompt('This is bayesian network:' + str(asia.edges()) + ' ' + PROMPT + ' Question: ' + question, model="phi4:latest")

            result = parse_input(response)
            # print('llm result\n', result, end='\n\n')

            # string_result = str(query_bn(asia, result["variables_of_interest"], result["evidence"]))
            # # print('bn result\n', string_result, end='\n\n')

            # answer = answer_this_prompt('Based on this result: ' + string_result + '. Give a short answer this question: ' + QUESTION)
            # # print('\nllm answer based on bn result:\n', answer)
            # print(result)
            rs.append(result)
            # print(answer)
      consistency_percentage, inconsistent_count, majority_result = check_consistency(rs)
      consistency_list.append(consistency_percentage)
      print(f"\nConsistency: {consistency_percentage:.2f}%")
      print(f"Inconsistent results: {inconsistent_count}")
      print(f"Majority result: {majority_result}")
      print('---------------------------------------------')
      print()

print(f'Average consistency: {sum(consistency_list) / len(consistency_list)}%')

Question 1 : What is the likelihood that someone has an X-ray if they have symptoms of dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 70.00%
Inconsistent results: 3
Majority result: {'variables_of_interest': ['xray'], 'evidence': {'dysp': 'yes'}}
---------------------------------------------

Question 2 : What's the likelihood that someone has pneumonia if they had an x-ray and didn't have dyspnea?
1 2 3 4 5 6 7 8 9 10 
Consistency: 60.00%
Inconsistent results: 4
Majority result: {'variables_of_interest': ['asia'], 'evidence': {'xray': 'yes', 'dysp': 'no'}}
---------------------------------------------

Question 3 : What's the likelihood that someone has pneumonia if their chest X-ray shows signs of disease?
1 2 3 4 5 6 7 8 9 10 
Consistency: 50.00%
Inconsistent results: 5
Majority result: {'variables_of_interest': ['asia'], 'evidence': {'xray': 'yes'}}
---------------------------------------------

Question 4 : What is the likelihood that a person with abnormal lung function also has dys